In [11]:
from email.mime import base
from re import A
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from cone_dataset import build_dataset, CustomDataset
from ConeSegmentation.CGNetPy1 import CGNet, CGNetEnd
import time
import pynvml

In [2]:
def infernece_test(model, model_end, dataset, num_tests=100, device=torch.device('cpu')):

    times1 = np.zeros([num_tests])
    times2 = np.zeros([num_tests])
    model.eval()
    model_end.eval()
    model.to(device)
    model_end.to(device)

    for idx, (image, mask) in enumerate(dataset):
        then = time.time()
        out = model(image.unsqueeze(0).to(device))
        dt1 = time.time() - then
        out = model_end(out)
        dt2 = time.time() - then

        times1[idx] = dt1
        times2[idx] = dt2

        if idx == num_tests - 1:
            break
    
    return np.mean(times1), np.mean(times2)

In [3]:
dataset = CustomDataset()
img, mask = dataset[0]

model = CGNet().cpu()
model_end = CGNetEnd().cpu()

state_dict = torch.load('weights/CGNetWeights_avmax')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [4]:
for _ in range(10):
    time1, time2 = infernece_test(model, model_end, dataset, num_tests=400, device=torch.device('cpu'))
    print(time1, time2)

0.133489471077919 0.1436373597383499
0.12945966839790343 0.13930539071559905
0.11633775115013123 0.12505071759223937
0.11617512881755829 0.12490913510322571
0.11610655307769775 0.12482508480548858
0.116098672747612 0.1248446649312973
0.11580618023872376 0.12463776707649231
0.11627399325370788 0.1250252443552017
0.11574151456356048 0.12448954224586486
0.1161336725950241 0.1248706716299057


In [5]:
def make_binary(array):
    base_array = np.zeros_like(array)
    base_array[array > 0.5] = 1

    return base_array

In [6]:
def accuracy_test(model, model_end, dataset, num_tests=100, device=torch.device('cpu')):

    accuacy = np.zeros([num_tests])
    model.eval()
    model_end.eval()
    model.to(device)
    model_end.to(device)

    for idx in range(num_tests):
# for idx, (image, mask) in enumerate(dataset):
        image, mask = dataset[np.random.randint(0, 1000)]
        out = model(image.unsqueeze(0).to(device))
        out = model_end(out)

        out = make_binary(out[0].cpu().detach().numpy())
        mask = make_binary(mask.cpu().detach().numpy())

        intersection = np.logical_and(out[0], mask)
        union = np.logical_or(out[0], mask)

        # plt.imshow(np.transpose(mask, (1, 2, 0)))
        # plt.savefig('mask.png')
        # plt.imshow(np.transpose(out, (1, 2, 0)))
        # plt.savefig('image.png')
        # plt.imshow(np.transpose(intersection, (1, 2, 0)))
        # plt.savefig('intersection.png')
        # plt.imshow(np.transpose(union, (1, 2, 0)))
        # plt.savefig('union.png')

        iou = np.sum(intersection).item() / np.sum(union).item() if np.sum(union).item() > 0 else 1
        accuacy[idx] = iou

        if idx == num_tests - 1:
            break
    
    return np.mean(accuacy)

In [7]:
for _ in range(10):
    accuracy = accuracy_test(model, model_end, dataset, num_tests=400, device=torch.device('cpu'))
    print(accuracy)

0.7088070919398217
0.7158786095237675
0.7010917823770063
0.700493504189148
0.7000981176235309
0.716750598344337
0.7004080227970627
0.7173411965288057
0.709395273884709
0.7143385885780072


In [12]:
def power_test(model, model_end, dataset, num_tests=100, device=torch.device('cuda')):

    torch.cuda.set_device(0)

    pynvml.nvmlInit()
    gpu = pynvml.nvmlDeviceGetHandleByIndex(0)
    name = pynvml.nvmlDeviceGetName(gpu)
#     device = name.decode("utf-8")

    powers = np.zeros([num_tests])
    model.eval()
    model_end.eval()
    model.to(device)
    model_end.to(device)

    for idx, (image, mask) in enumerate(dataset):
        out = model(image.unsqueeze(0).to(device))
        out = model_end(out)
        power_usage = pynvml.nvmlDeviceGetPowerUsage(gpu) / 1000

        powers[idx] = power_usage

        if idx == num_tests - 1:
            break
    
    return np.mean(powers)

In [13]:
for _ in range(10):
    power = power_test(model, model_end, dataset, num_tests=10, device=torch.device('cpu'))
    print(power)

6.157
6.0514
6.061100000000001
6.1088
6.08
6.0416
6.0707
6.0512
6.0553
6.118099999999999


In [14]:
for _ in range(10):
    time1, time2 = infernece_test(model, model_end, dataset, num_tests=400, device=torch.device('cuda'))
    print(time1, time2)

0.014658648371696472 0.01473906934261322
0.01407943308353424 0.014153441786766053
0.014710987210273743 0.014789448380470275
0.014114563465118407 0.014188040494918824
0.014752710461616516 0.014833747744560241
0.014517396688461304 0.014594383239746093
0.01442046046257019 0.014496827721595764
0.01422640860080719 0.01430175542831421
0.014283732771873473 0.01435928225517273
0.014316163063049316 0.014392366409301758


In [15]:
for _ in range(10):
    accuracy = accuracy_test(model, model_end, dataset, num_tests=400, device=torch.device('cuda'))
    print(accuracy)

0.7235758868843862
0.7130505302498232
0.7033104639086138
0.7135488398482818
0.7007474485634114
0.717001372012055
0.7207180928450996
0.7055738054648337
0.7163707857737447
0.7179191184928275


In [16]:
for _ in range(10):
    power = power_test(model, model_end, dataset, num_tests=10, device=torch.device('cuda'))
    print(power)

48.4956
48.1876
48.5463
48.362
47.92229999999999
48.390699999999995
47.5332
48.0295
48.0983
46.5888
